<a href="https://colab.research.google.com/github/orgoca/W207FinalProject/blob/main/Project2EDA_JH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 2 Baseline
Quora Project



# Load Data and Import Libraries

First, we load all data and necessary libraries. We split the `train` data into `miniTrain` and `miniTest` dataframes.



In [1]:
# Code to read csv file from Google Drive into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
#Import Libraries
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from sklearn_pandas import DataFrameMapper
import csv
import random

In [3]:
#Load test DF
testLink = 'https://drive.google.com/file/d/1chKVOcXwNzGRe_hRxpGlq18NVaSRM8X9/view?usp=sharing'
idTest = testLink.split('/')[-2]
downloaded = drive.CreateFile({'id':idTest}) 
downloaded.GetContentFile('test.csv')  

In [4]:
test = pd.read_csv('test.csv')

In [5]:
#Load train DF
trainLink = 'https://drive.google.com/file/d/1fP-AWiUGLjLkL0c_u67HOUFmHOMO7qm2/view?usp=sharing'
idTrain = trainLink.split('/')[-2]
downloaded = drive.CreateFile({'id':idTrain}) 
downloaded.GetContentFile('train.csv')  

In [6]:
train = pd.read_csv('train.csv')

In [8]:
#Split Training Data into MiniTrain and MiniTest (Test data has no labels so impossible to measure error from it)
random.seed(10)
miniTrain, miniTest = train_test_split(train, test_size=0.2)

miniTrain.to_csv('miniTrain.csv', index=False)

miniTest.to_csv('miniTest.csv', index=False)

In [9]:
#Even with a seed the randomization is messing up predictions since miniTest will not find the same questions further down, 
#so rather load a static version from a csv instead.

miniTrain = pd.read_csv('miniTrain.csv')

miniTest = pd.read_csv('miniTest.csv')

# Vocabulary

Now we can instantiate a `CountVectorizer` to generate a vocabulary based on our `miniTrain` data.

In [10]:
#Create a vectorizer, create question and target lists, and vectorize train questions
vectorizer = CountVectorizer()
trainQuestions = miniTrain.question_text.tolist()
trainLabels = miniTrain.target.tolist()
vTrain = vectorizer.fit_transform(trainQuestions)

In [11]:
#What is the vocabulary size, and create a bag of words
print("vocab size: ", vTrain.shape)
bagOfWords = vectorizer.vocabulary_
len(bagOfWords)

vocab size:  (1044897, 173334)


173334

In [12]:
#Create a logistic regression model and fit to train data
lr_clf = LogisticRegression(C=1)
lr_clf.fit(vTrain, trainLabels)

train_predict = lr_clf.predict(vTrain)
train_predict

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
lr_clf.predict_proba(vTrain)

array([[0.98058194, 0.01941806],
       [0.68642571, 0.31357429],
       [0.9984996 , 0.0015004 ],
       ...,
       [0.99433734, 0.00566266],
       [0.71856399, 0.28143601],
       [0.87453048, 0.12546952]])

In [ ]:
#Give this a try with your own questions, it's good fun!
#Try an insincere question. Insincere questions should get a higher probability score to legitimate questions.
testQuestionInsincere1 = "Why are Indians a superior race to Pakistanis?"

vdev = vectorizer.transform([testQuestionInsincere1.lower()])

lr_clf.predict_proba(vdev)[0][1]

0.948091663155399

In [ ]:
#Try a sincere question. Sincere questions should get a lower probability score ideally close to zero.
testQuestionInsincere2 = "What are the origins of the conflict between Indians and Pakistanis?"
vdev = vectorizer.transform([testQuestionInsincere2.lower()])

lr_clf.predict_proba(vdev)[0][1]

0.20721163916364985

In [ ]:
# #DON'T run this section. It takes too long. I've produced a CSV from the output and you can run from there by loading such CSV in the next block
# #Let's predict probability for each question in the miniTest dataframe
columnNames = ['question_text', 'pred']
predictDF = pd.DataFrame(columns = columnNames)

# predictDF["question_text"] = miniTest.question_text
# predictDF["pred"] = (predictDF["question_text"].str.lower() \
#                       .apply(list) \
#                       .apply(vectorizer.transform) \
#                       .apply(lr_clf.predict_proba) \
#                       .apply(lambda x: x[0][1]) )

# # predictDF["vdev"] = predictDF.q_lower.apply(list).apply(vectorizer.transform)
# # predictDF["pred"] = predictDF.vdev.apply(lr_clf.predict_proba).apply(lambda x: x[0][1])
# predictDF.head()

for question in miniTest.question_text: 
  vdev = vectorizer.transform([question.lower()])
  pred = lr_clf.predict_proba(vdev)[0][1]
  predictDF = predictDF.append({'question_text':question,'pred': pred}, ignore_index=True)

predictDF.to_csv('predictDF.csv', index=False)

In [ ]:
## DONT RUN THIS - JUST EXPERIMENTING

# columnNames = ['question_text', 'pred']
# predictDF = pd.DataFrame(columns = columnNames)

# predictDF["question_text"] = miniTest.question_text
# predictDF["q_lower"] = predictDF["question_text"].str.lower()



# mapper = DataFrameMapper([
#   ('q_lower', vectorizer)
# ])

# predictDF = mapper.transform(predictDF)
# # predictDF.head()

TypeError: ignored

In [ ]:
predictDF = pd.read_csv('predictDF.csv')

predictDF

,question_text,pred
0,"If you are a marketplace, is it legal to use y...",0.008937
1,"In a vacuum, does a water drop fall free and e...",0.004993
2,What do you do when the girl you have started ...,0.013679
3,Was MHTCET engineering conducted before 2016?,0.008991
4,Is there some inner logic built in the universe?,0.005960
...,...,...
261220,What can be the best strategy for building a b...,0.001013
261221,Can you enter Mumbai domestic airport 8-9 hour...,0.006073
261222,What is a product old people need and could bu...,0.007697
261223,"Do Pakistanis know the fact that Jinnah, the v...",0.129631


In [ ]:
miniTest

,qid,question_text,target
0,1cd82e247425bc2f0ccc,"If you are a marketplace, is it legal to use y...",0
1,badeae238717453413fd,"In a vacuum, does a water drop fall free and e...",0
2,a07cf954038ccf4d9ba9,What do you do when the girl you have started ...,0
3,8230e89677a6cd7a460d,Was MHTCET engineering conducted before 2016?,0
4,4768d464d96b8d60e430,Is there some inner logic built in the universe?,0
...,...,...,...
261220,b1c80f3f0751ca815772,What can be the best strategy for building a b...,0
261221,9f9e42cf1dd98b845c3d,Can you enter Mumbai domestic airport 8-9 hour...,0
261222,56c11e63f3a8c5b162b7,What is a product old people need and could bu...,0
261223,ea903eefd8ddd3c0c6a3,"Do Pakistanis know the fact that Jinnah, the v...",1


In [ ]:
# #Let's see some nasty questions
# nastyQuestions = predictDF[predictDF.pred > .9]

# nastyQuestions.to_csv('nastyQuestions.csv', index=False)

In [ ]:
#Merge predict DF to miniTestDF to produce prediction metrics

summaryDF = pd.merge(predictDF,miniTest,on='question_text',how='left')

# summaryDF.to_csv('summaryDF.csv', index=False)

summaryDF

,question_text,pred,qid,target
0,"If you are a marketplace, is it legal to use y...",0.008937,1cd82e247425bc2f0ccc,0
1,"In a vacuum, does a water drop fall free and e...",0.004993,badeae238717453413fd,0
2,What do you do when the girl you have started ...,0.013679,a07cf954038ccf4d9ba9,0
3,Was MHTCET engineering conducted before 2016?,0.008991,8230e89677a6cd7a460d,0
4,Is there some inner logic built in the universe?,0.005960,4768d464d96b8d60e430,0
...,...,...,...,...
261220,What can be the best strategy for building a b...,0.001013,b1c80f3f0751ca815772,0
261221,Can you enter Mumbai domestic airport 8-9 hour...,0.006073,9f9e42cf1dd98b845c3d,0
261222,What is a product old people need and could bu...,0.007697,56c11e63f3a8c5b162b7,0
261223,"Do Pakistanis know the fact that Jinnah, the v...",0.129631,ea903eefd8ddd3c0c6a3,1


In [ ]:
confusionMatrixDF = summaryDF[['pred', 'target']]

#Tried -by hand- values between 0.5:1.0 and 0.2 produced the best F1 score

confusionMatrixDF['pred'] = np.where(confusionMatrixDF['pred'] > 0.2, 1, 0)

confusionMatrixDF

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,pred,target
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
261220,0,0
261221,0,0
261222,0,0
261223,0,1


In [ ]:
#Compute baseline F1 Score (and other metrics)
confusionMatrixDF
 
print('Accuracy:', accuracy_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('F1 score:', f1_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('Recall:', recall_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('Precision:', precision_score(confusionMatrixDF.target, confusionMatrixDF.pred))
print('\n Clasification report:\n', classification_report(confusionMatrixDF.target,confusionMatrixDF.pred))
print('\n Confussion matrix:\n',confusion_matrix(confusionMatrixDF.target, confusionMatrixDF.pred))

Accuracy: 0.950502440424921
F1 score: 0.6518390866497926
Recall: 0.7448157036490062
Precision: 0.57949921003495

 Clasification report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97    244974
           1       0.58      0.74      0.65     16251

    accuracy                           0.95    261225
   macro avg       0.78      0.85      0.81    261225
weighted avg       0.96      0.95      0.95    261225


 Confussion matrix:
 [[236191   8783]
 [  4147  12104]]
